In [1]:
import numpy as np
import os

## 基于用户的

In [2]:
def readFile(filename):
    #files = open(filename, "r", encoding="utf-8")
    # 如果读取不成功试一下
    files = open(filename, "r", encoding="iso-8859-15")
    data = []
    for line in files.readlines():
        item = line.strip().split("::")
        data.append(item)
    return data

In [3]:
basepath = r'D:\jupyterworkplace\Tutorials\machine learning\ml-1m\ml-1m'

In [4]:
movies = readFile(os.path.join(basepath,"movies.dat"))
ratings = readFile(os.path.join(basepath,"ratings.dat"))

In [5]:
#movies的数据格式
movies[:5]

[['1', 'Toy Story (1995)', "Animation|Children's|Comedy"],
 ['2', 'Jumanji (1995)', "Adventure|Children's|Fantasy"],
 ['3', 'Grumpier Old Men (1995)', 'Comedy|Romance'],
 ['4', 'Waiting to Exhale (1995)', 'Comedy|Drama'],
 ['5', 'Father of the Bride Part II (1995)', 'Comedy']]

In [6]:
#rating的数据格式
ratings[:5]

[['1', '1193', '5', '978300760'],
 ['1', '661', '3', '978302109'],
 ['1', '914', '3', '978301968'],
 ['1', '3408', '4', '978300275'],
 ['1', '2355', '5', '978824291']]

In [8]:
def formatRate(ratings):
    userDict = {}
    ItemUser = {}
    for i in ratings:
        # 评分最高为5 除以5 进行数据归一化
        temp = (i[1], float(i[2]) / 5)
        # 计算userDict {'1':[(1,5),(2,5)...],'2':[...]...}
        if(i[0] in userDict):
            userDict[i[0]].append(temp)
        else:
            userDict[i[0]] = [temp]
        # 计算ItemUser {'1',[1,2,3..],...}
        if(i[1] in ItemUser):
            ItemUser[i[1]].append(i[0])
        else:
            ItemUser[i[1]] = [i[0]]
    return userDict , ItemUser

In [9]:
userDict , ItemUser = formatRate(ratings)

In [27]:
#userDict是一个字典 包含每个用户评价的电影的信息
#下面选取第67个用户所有评价的电影中的前5个（没有顺序之分）
userDict['67'][:5]

[('1250', 0.6), ('1185', 1.0), ('587', 0.4), ('2135', 0.8), ('1408', 0.6)]

In [29]:
#ItemUser也是一个字典 包含每部电影呗哪些用户评价过
#下面选取第188部电影所有评价用户中的前5个（也没有顺序之分）
ItemUser['188'][:5]

['53', '163', '195', '223', '321']

In [34]:
#计算两个用户之间的交并集合
#返回的是字典 user 
# user = {movieID：[userId的评分，l的评分]}
def formatuserDict(userId, l):
    user = {}
    for i in userDict[userId]:
        user[i[0]] = [i[1], 0]
    for j in userDict[l]:
        if(j[0] not in user):
            user[j[0]] = [0, j[1]]
        else:
            user[j[0]][1] = j[1]
    return user

In [37]:
#计算用户1和2之间所评价的电影的关系
user = formatuserDict('1','2')

In [40]:
#计算两个用户之间的相似度
def getCost(userId, l):
    # step1：获取用户userId和l评分电影的并集
    # user={'电影ID'：[userId的评分，l的评分]} 没有评分为0
    user = formatuserDict(userId, l)
    x = 0.0
    y = 0.0
    z = 0.0
    for k, v in user.items():
        x += float(v[0]) * float(v[0])
        y += float(v[1]) * float(v[1])
        z += float(v[0]) * float(v[1])
    if(z == 0.0):
        return 0
    return z / np.sqrt(x * y)

In [41]:
getCost('1','2')

0.096381530497751625

In [42]:
'''
计算一个用户最相近的k个用户
step1: 获取userId评分的电影都有那些用户也评过分（求一个并集的概念）
step2：求所有用户的相似度 取前k个
'''
def getNearestNeighbor(userId,k):
    userDict , ItemUser = formatRate(ratings)
    neighbors = []
    neighbors_dist = []
    # 获取userId评分的电影都有那些用户也评过分
    for i in userDict[userId]:
        for j in ItemUser[i[0]]:
            if(j != userId and j not in neighbors):
                neighbors.append(j)
    # 计算这些用户与userId的相似度并排序
    for i in neighbors:
        dist = getCost(userId, i)
        neighbors_dist.append([dist, i])
    # 排序默认是升序，reverse=True表示降序
    neighbors_dist.sort(reverse=True)
    neighbors_dist = neighbors_dist[:k]
    return neighbors_dist

In [45]:
#获取用户ID1的最相似的5个用户的ID
neighbors=getNearestNeighbor('1',5)

In [46]:
neighbors

[[0.41211706435523543, '5343'],
 [0.41189862329150367, '5190'],
 [0.39211038220634808, '1481'],
 [0.38659663728150834, '1283'],
 [0.36089838113994654, '5705']]

In [59]:
def getrecommandList(userId,k):
    userDict , ItemUser = formatRate(ratings)
    neighbors=getNearestNeighbor(userId,k)
    n = len(userDict[userId])
    recommandList = []
    # 建立推荐字典
    recommandDict = {}
    #用户已近评价的电影
    evaluated = []
    for i in userDict[userId]:
        evaluated.append(i[0])
    for neighbor in neighbors:
        movies = userDict[neighbor[1]]
        for movie in movies:
            if movie[0] not in evaluated:       
                if(movie[0] in recommandDict):
                    recommandDict[movie[0]] += neighbor[0]
                else:
                    recommandDict[movie[0]] = neighbor[0]
    # 建立推荐列表
    for key in recommandDict:
        recommandList.append([recommandDict[key], key])
    recommandList.sort(reverse=True)
    recommandList = recommandList[:n]
    return recommandList

In [60]:
recommand = getrecommandList('1',5)

## 基于物品的

In [11]:
print(len(ItemUser),len(userDict))

3706 6040


In [20]:
tongxian_matrix = np.zeros([3707,3707])

In [44]:
for i in range(1,3707):
    for j in range(1,3707):
        if i==j:
            continue
        if j>i:
            count = len( set( ItemUser.get(str(i),0) if ItemUser.get(str(i),0) else []) & 
                       set( ItemUser.get(str(j),0) if ItemUser.get(str(j),0) else []))
            tongxian_matrix[i][j] = count
        else:
            tongxian_matrix[i][j] = tongxian_matrix[j][i]       

In [46]:
tongxian_matrix

array([[   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,  491., ...,  396.,  225.,  231.],
       [   0.,  491.,    0., ...,  237.,  163.,  129.],
       ..., 
       [   0.,  396.,  237., ...,    0.,  167.,  187.],
       [   0.,  225.,  163., ...,  167.,    0.,   99.],
       [   0.,  231.,  129., ...,  187.,   99.,    0.]])

In [47]:
#去除0行0列的值
tongxian_array = tongxian_matrix[1:,1:]

In [48]:
tongxian_array

array([[   0.,  491.,  287., ...,  396.,  225.,  231.],
       [ 491.,    0.,  153., ...,  237.,  163.,  129.],
       [ 287.,  153.,    0., ...,  125.,  124.,   63.],
       ..., 
       [ 396.,  237.,  125., ...,    0.,  167.,  187.],
       [ 225.,  163.,  124., ...,  167.,    0.,   99.],
       [ 231.,  129.,   63., ...,  187.,   99.,    0.]])

In [51]:
#获取评分矩阵
# 以用户 1 为例
rating_1 = np.zeros(3707)
for i in userDict['1']:
    rating_1[int(i[0])] = i[1]

In [54]:
rating1 = rating_1[1:]

In [55]:
rating1.shape

(3706,)

In [62]:
movie1_to_other  = np.dot(tongxian_array,rating1)

In [63]:
index = np.argsort(movie1_to_other)

In [64]:
index[:10]

array([ 692,  767, 1696, 1697, 1699, 1071, 1704, 2318, 1705, 1707], dtype=int64)